# Задача 1.

Для начала давайте проанализируем, насколько быстро растёт аудитория нашего сервиса, и посмотрим на динамику числа пользователей и курьеров. 

Задание:

Для каждого дня, представленного в таблицах user_actions и courier_actions, рассчитайте следующие показатели:

Число новых пользователей.
Число новых курьеров.
Общее число пользователей на текущий день.
Общее число курьеров на текущий день.
Колонки с показателями назовите соответственно new_users, new_couriers, total_users, total_couriers. Колонку с датами назовите date. Проследите за тем, чтобы показатели были выражены целыми числами. Результат должен быть отсортирован по возрастанию даты.

Поля в результирующей таблице: date, new_users, new_couriers, total_users, total_couriers

Пояснение: 

Новыми будем считать тех пользователей и курьеров, которые в данный день совершили своё первое действие в нашем сервисе. Общее число пользователей/курьеров на текущий день — это результат сложения числа новых пользователей/курьеров в текущий день со значениями аналогичного показателя всех предыдущих дней.

```sql
SELECT date,
       new_users,
       new_couriers,
       sum(new_users) OVER(ORDER BY date)::int as total_users,
       sum(new_couriers) OVER(ORDER BY date)::int as total_couriers
FROM   ((SELECT count(user_id) as new_users,
                date
         FROM   (SELECT user_id,
                        min(time)::date date
                 FROM   user_actions
                 GROUP BY user_id) as t0
         GROUP BY date
         ORDER BY date) as t1
    LEFT JOIN (SELECT count(courier_id) as new_couriers,
                      date
               FROM   (SELECT courier_id,
                              min(time)::date date
                       FROM   courier_actions
                       GROUP BY courier_id) as t0
               GROUP BY date
               ORDER BY date) as t2 using(date)) as t12
ORDER BY date
```

# Задача 2.

Анализируя динамику показателей из предыдущего задания, вы могли заметить, что сравнивать абсолютные значения не очень удобно. Давайте посчитаем динамику показателей в относительных величинах.

Задание:

Дополните запрос из предыдущего задания и теперь для каждого дня, представленного в таблицах user_actions и courier_actions, дополнительно рассчитайте следующие показатели:

Прирост числа новых пользователей.
Прирост числа новых курьеров.
Прирост общего числа пользователей.
Прирост общего числа курьеров.
Показатели, рассчитанные на предыдущем шаге, также включите в результирующую таблицу.

Колонки с новыми показателями назовите соответственно new_users_change, new_couriers_change, total_users_growth, total_couriers_growth. Колонку с датами назовите date.

Все показатели прироста считайте в процентах относительно значений в предыдущий день. При расчёте показателей округляйте значения до двух знаков после запятой.

Результирующая таблица должна быть отсортирована по возрастанию даты.

Поля в результирующей таблице: 

date, new_users, new_couriers, total_users, total_couriers, 

new_users_change, new_couriers_change, total_users_growth, total_couriers_growth

Пояснение:

Не забывайте при делении заранее приводить значения к нужному типу данных. Пропущенные значения приростов для самой первой даты не заполняйте — просто оставьте поля в этой строке пустыми.

```sql
SELECT date,
       new_users,
       new_couriers,
       total_users,
       total_couriers,
       new_users_change,
       new_couriers_change,
       round(total_users::decimal / lag(total_users, 1) OVER(ORDER BY date) * 100 - 100,
             2) as total_users_growth,
       round(total_couriers::decimal / lag(total_couriers, 1) OVER(ORDER BY date) * 100 - 100,
             2) as total_couriers_growth
FROM   (SELECT date,
               new_users,
               new_couriers,
               sum(new_users) OVER(ORDER BY date)::int as total_users,
               sum(new_couriers) OVER(ORDER BY date)::int as total_couriers,
               round(new_users::decimal / lag(new_users, 1) OVER(ORDER BY date) * 100 - 100,
                     2) as new_users_change,
               round(new_couriers::decimal / lag(new_couriers, 1) OVER(ORDER BY date) * 100 - 100,
                     2) as new_couriers_change
        FROM   ((SELECT count(user_id) as new_users,
                        date
                 FROM   (SELECT user_id,
                                min(time)::date date
                         FROM   user_actions
                         GROUP BY user_id) as t0
                 GROUP BY date
                 ORDER BY date) as t1
            LEFT JOIN (SELECT count(courier_id) as new_couriers,
                              date
                       FROM   (SELECT courier_id,
                                      min(time)::date date
                               FROM   courier_actions
                               GROUP BY courier_id) as t0
                       GROUP BY date
                       ORDER BY date) as t2 using(date)) as t12
        ORDER BY date) as t121
ORDER BY date
```

# Задача 3.

Теперь предлагаем вам посмотреть на нашу аудиторию немного под другим углом — давайте посчитаем не просто всех пользователей, а именно ту часть, которая оформляет и оплачивает заказы в нашем сервисе. Заодно выясним, какую долю платящие пользователи составляют от их общего числа.

Задание:

Для каждого дня, представленного в таблицах user_actions и courier_actions, рассчитайте следующие показатели:

Число платящих пользователей.
Число активных курьеров.
Долю платящих пользователей в общем числе пользователей на текущий день.
Долю активных курьеров в общем числе курьеров на текущий день.
Колонки с показателями назовите соответственно paying_users, active_couriers, paying_users_share, active_couriers_share. Колонку с датами назовите date. Проследите за тем, чтобы абсолютные показатели были выражены целыми числами. Все показатели долей необходимо выразить в процентах. При их расчёте округляйте значения до двух знаков после запятой.

Результат должен быть отсортирован по возрастанию даты. 

Поля в результирующей таблице: date, paying_users, active_couriers, paying_users_share, active_couriers_share

Пояснение: 

Платящими будем считать тех пользователей, которые в данный день оформили хотя бы один заказ, который в дальнейшем не был отменен.

Курьеров будем считать активными, если в данный день они приняли хотя бы один заказ, который был доставлен (возможно, уже на следующий день), или доставили любой заказ.

Общее число пользователей/курьеров на текущий день — это по-прежнему результат сложения числа новых пользователей/курьеров в текущий день со значениями аналогичного показателя всех предыдущих дней. Мы считали этот показатель на предыдущих шагах.

Не забывайте при делении заранее приводить значения к нужному типу данных. 

```sql
SELECT date,
       paying_users,
       active_couriers,
       round(paying_users::decimal / total_users * 100, 2) as paying_users_share,
       round(active_couriers::decimal / total_couriers * 100, 2) as active_couriers_share
FROM   (SELECT date,
               total_users,
               total_couriers,
               paying_users,
               active_couriers
        FROM   (SELECT date,
                       paying_users,
                       active_couriers,
                       sum(new_users) OVER(ORDER BY date)::int as total_users,
                       sum(new_couriers) OVER(ORDER BY date)::int as total_couriers
                FROM   ((SELECT count(distinct user_id) as new_users,
                                min_time as date
                         FROM   (SELECT user_id,
                                        min(time)::date as min_time
                                 FROM   user_actions
                                 GROUP BY user_id) as q1
                         GROUP BY date) as t2
                    LEFT JOIN (SELECT count(distinct courier_id) as new_couriers,
                                      min_time as date
                               FROM   (SELECT courier_id,
                                              min(time)::date as min_time
                                       FROM   courier_actions
                                       GROUP BY courier_id) as q2
                               GROUP BY date) as t1 using(date)) as p1
                    LEFT JOIN ((SELECT count(distinct user_id) as paying_users,
                                       min_time as date
                                FROM   (SELECT user_id,
                                               time::date as min_time
                                        FROM   user_actions
                                        WHERE  order_id not in (SELECT order_id
                                                                FROM   user_actions
                                                                WHERE  action = 'cancel_order')) as q1
                    LEFT JOIN (SELECT count(distinct courier_id) as active_couriers,
                                      min_time as date
                               FROM   (SELECT courier_id,
                                              time::date as min_time
                                       FROM   courier_actions
                                       WHERE  order_id in (SELECT order_id
                                                           FROM   courier_actions
                                                           WHERE  action = 'deliver_order')) as q2
                               GROUP BY date) as t1 using(date)) as p2 using(date)) as pp1) as ppp1
ORDER BY date
```

# Задача 4.

Давайте подробнее остановимся на платящих пользователях, копнём немного глубже и выясним, как много платящих пользователей совершают более одного заказа в день. В конце концов нам важно понимать, как в большинстве своём ведут себя наши пользователи — они заходят в приложение, чтобы сделать всего один заказ, или же наш сервис настолько хорош, что они готовы пользоваться им несколько раз в день.

Задание:

Для каждого дня, представленного в таблице user_actions, рассчитайте следующие показатели:

Долю пользователей, сделавших в этот день всего один заказ, в общем количестве платящих пользователей.
Долю пользователей, сделавших в этот день несколько заказов, в общем количестве платящих пользователей.
Колонки с показателями назовите соответственно single_order_users_share, several_orders_users_share. Колонку с датами назовите date. Все показатели с долями необходимо выразить в процентах. При расчёте долей округляйте значения до двух знаков после запятой.

Результат должен быть отсортирован по возрастанию даты.

Поля в результирующей таблице: date, single_order_users_share, several_orders_users_share

Пояснение: 

Платящими по-прежнему считаем тех пользователей, которые в данный день оформили (и не отменили) хотя бы один заказ.

Не забывайте при делении заранее приводить значения к нужному типу данных.

```sql
SELECT date,
       round(one_order_user::decimal / paying_users * 100,
             2) as single_order_users_share,
       round(more_order_user::decimal / paying_users * 100,
             2) as several_orders_users_share
FROM   ((SELECT date,
                count(one_order_user) as one_order_user,
                count(more_order_user) as more_order_user
         FROM   (SELECT case when order_count = 1 then count(user_id) OVER(PARTITION BY date) end as one_order_user ,
                        case when order_count > 1 then count(user_id) OVER(PARTITION BY date) end as more_order_user ,
                        date
                 FROM   (SELECT count(order_id) as order_count,
                                min_time as date,
                                user_id
                         FROM   (SELECT order_id,
                                        min(time)::date as min_time,
                                        user_id
                                 FROM   user_actions
                                 WHERE  order_id not in (SELECT order_id
                                                         FROM   user_actions
                                                         WHERE  action = 'cancel_order')
                                 GROUP BY order_id, user_id) as p0
                         GROUP BY date, user_id) as p1) as q1
         GROUP BY date) as qqqqq1
    LEFT JOIN (SELECT min_time as date,
                      count(distinct user_id) as paying_users
               FROM   (SELECT user_id,
                              time::date as min_time
                       FROM   user_actions
                       WHERE  order_id not in (SELECT order_id
                                               FROM   user_actions
                                               WHERE  action = 'cancel_order')) as q1
               GROUP BY min_time) as qqqqq2 using(date)) as itog_from
ORDER BY date
```

# *Задача 5.

Продолжим изучать наш сервис и рассчитаем несколько показателей, связанных с заказами.

Задание:

Для каждого дня, представленного в таблице user_actions, рассчитайте следующие показатели:

Общее число заказов.
Число первых заказов (заказов, сделанных пользователями впервые).
Число заказов новых пользователей (заказов, сделанных пользователями в тот же день, когда они впервые воспользовались сервисом).
Долю первых заказов в общем числе заказов (долю п.2 в п.1).
Долю заказов новых пользователей в общем числе заказов (долю п.3 в п.1).
Колонки с показателями назовите соответственно orders, first_orders, new_users_orders, first_orders_share, new_users_orders_share. Колонку с датами назовите date. Проследите за тем, чтобы во всех случаях количество заказов было выражено целым числом. Все показатели с долями необходимо выразить в процентах. При расчёте долей округляйте значения до двух знаков после запятой.

Результат должен быть отсортирован по возрастанию даты.

Поля в результирующей таблице: date, orders, first_orders, new_users_orders, first_orders_share, new_users_orders_share

Пояснение: 

При расчёте числа первых заказов учитывайте, что у каждого пользователя может быть всего один первый заказ (что вполне логично).

При расчёте числа заказов новых пользователей учитывайте, что в свой первый день каждый новый пользователь мог как совершить сразу несколько заказов, так и не совершить ни одного.

Во всех случаях при расчёте числа заказов учитывайте только фактически совершённые заказы, отменённые заказы не учитывайте.

Не забывайте при делении заранее приводить значения к нужному типу данных.

```sql
SELECT date,
       orders,
       first_orders,
       new_users_orders,
       round(first_orders::decimal / orders * 100, 2) as first_orders_share,
       round(new_users_orders::decimal / orders * 100, 2) as new_users_orders_share
FROM   ((SELECT date,
                orders,
                first_orders
         FROM   ((SELECT count(order_id) as orders,
                         time::date as date
                  FROM   user_actions
                  WHERE  order_id not in (SELECT order_id
                                          FROM   user_actions
                                          WHERE  action = 'cancel_order')
                  GROUP BY date) as t1
             LEFT JOIN (SELECT count(distinct user_id) as first_orders,
                               min_time::date as date
                        FROM   (SELECT user_id,
                                       min(time) as min_time
                                FROM   user_actions
                                WHERE  order_id not in (SELECT order_id
                                                        FROM   user_actions
                                                        WHERE  action = 'cancel_order')
                                GROUP BY user_id) as q1
                        GROUP BY date) as t2 using(date)) as t12) as t121
    LEFT JOIN (SELECT start_date as date,
                      sum(orders)::int as new_users_orders
               FROM   (SELECT e1.user_id,
                              e1.start_date,
                              coalesce(e2.orders, 0) as orders
                       FROM   (SELECT user_id,
                                      min(time::date) as start_date
                               FROM   user_actions
                               GROUP BY user_id) as e1
                           LEFT JOIN (SELECT user_id,
                                             time::date as date,
                                             count(distinct order_id) as orders
                                      FROM   user_actions
                                      WHERE  order_id not in (SELECT order_id
                                                              FROM   user_actions
                                                              WHERE  action = 'cancel_order')
                                      GROUP BY user_id, date) as e2
                               ON e1.user_id = e2.user_id and
                                  e1.start_date = e2.date) as e12
               GROUP BY date) as e121 using(date)) as te1
ORDER BY date
```

# Задача 6.

Теперь давайте попробуем примерно оценить нагрузку на наших курьеров и узнаем, сколько в среднем заказов и пользователей приходится на каждого из них.

Задание:

На основе данных в таблицах user_actions, courier_actions и orders для каждого дня рассчитайте следующие показатели:

Число платящих пользователей на одного активного курьера.
Число заказов на одного активного курьера.
Колонки с показателями назовите соответственно users_per_courier и orders_per_courier. Колонку с датами назовите date. При расчёте показателей округляйте значения до двух знаков после запятой.

Результирующая таблица должна быть отсортирована по возрастанию даты.

Поля в результирующей таблице: date, users_per_courier, orders_per_courier

Пояснение:

Платящими по-прежнему считаем тех пользователей, которые в данный день оформили хотя бы один заказ, который в дальнейшем не был отменен.

Курьеров считаем активными, если в данный день они приняли хотя бы один заказ, который был доставлен (возможно, уже на следующий день), или доставили любой заказ.

В расчётах учитывайте только неотменённые заказы. 

Не забывайте при делении заранее приводить значения к нужному типу данных.

```sql
SELECT date,
       round(using_count::decimal / couriers_count, 2) as users_per_courier,
       round(orders_count::decimal / couriers_count, 2) as orders_per_courier
FROM   ((SELECT count(distinct user_id) as using_count,
                count(order_id) filter(WHERE action = 'create_order') as orders_count,
                time::date as date
         FROM   user_actions
         WHERE  order_id not in (SELECT order_id
                                 FROM   user_actions
                                 WHERE  action = 'cancel_order')
         GROUP BY date) as t1
    LEFT JOIN (SELECT count(distinct courier_id) as couriers_count,
                      time::date as date
               FROM   courier_actions
               WHERE  order_id not in (SELECT order_id
                                       FROM   user_actions
                                       WHERE  action = 'cancel_order')
               GROUP BY date) as t2 using(date)) as t12
ORDER BY date
```

# Задача 7.

Давайте рассчитаем ещё один полезный показатель, характеризующий качество работы курьеров.

Задание:

На основе данных в таблице courier_actions для каждого дня рассчитайте, за сколько минут в среднем курьеры доставляли свои заказы.

Колонку с показателем назовите minutes_to_deliver. Колонку с датами назовите date. При расчёте среднего времени доставки округляйте количество минут до целых значений. Учитывайте только доставленные заказы, отменённые заказы не учитывайте.

Результирующая таблица должна быть отсортирована по возрастанию даты.

Поля в результирующей таблице: date, minutes_to_deliver

Пояснение:

Для решения задачи вам, возможно, придётся вспомнить, как определить количество минут, содержащихся в интервале времени. Мы уже решали похожую задачу на этом шаге.

Некоторые заказы оформляют в один день, а доставляют уже на следующий. При расчёте среднего времени доставки в качестве дней, для которых считать среднее, используйте дни фактической доставки заказов.



```sql
SELECT date,
       avg(extract(epoch
FROM   deliver_time - accept_time)/60)::integer as minutes_to_deliver
FROM   ((SELECT min(time) as accept_time,
                max(time) as deliver_time,
                order_id
         FROM   courier_actions
         WHERE  order_id not in (SELECT order_id
                                 FROM   user_actions
                                 WHERE  action = 'cancel_order')
         GROUP BY order_id
         ORDER BY order_id) as t1
    LEFT JOIN (SELECT max(time::date) as date,
                      order_id
               FROM   user_actions
               WHERE  order_id not in (SELECT order_id
                                       FROM   user_actions
                                       WHERE  action = 'cancel_order')
               GROUP BY order_id
               ORDER BY order_id) as t2 using(order_id)) as t12
GROUP BY date
ORDER BY date
```

# Задача 8.
И наконец, давайте оценим почасовую нагрузку на наш сервис, выясним, в какие часы пользователи оформляют больше всего заказов, и заодно проанализируем, как изменяется доля отмен в зависимости от времени оформления заказа.

Задача:

На основе данных в таблице orders для каждого часа в сутках рассчитайте следующие показатели:

Число успешных (доставленных) заказов.
Число отменённых заказов.
Долю отменённых заказов в общем числе заказов (cancel rate).
Колонки с показателями назовите соответственно successful_orders, canceled_orders, cancel_rate. Колонку с часом оформления заказа назовите hour. При расчёте доли отменённых заказов округляйте значения до трёх знаков после запятой.

Результирующая таблица должна быть отсортирована по возрастанию колонки с часом оформления заказа.

Поля в результирующей таблице: hour, successful_orders, canceled_orders, cancel_rate

```sql
SELECT hour,
       successful_orders,
       canceled_orders,
       round(canceled_orders::decimal / (successful_orders + canceled_orders),
             3) as cancel_rate
FROM   ((SELECT count(order_id) as successful_orders,
                date_part('hour', time)::int as hour
         FROM   user_actions
         WHERE  order_id not in (SELECT order_id
                                 FROM   user_actions
                                 WHERE  action = 'cancel_order')
         GROUP BY hour
         ORDER BY hour) as t1
    LEFT JOIN (SELECT count(distinct order_id) filter(WHERE order_id in (SELECT order_id
                                                                                        FROM   user_actions
                                                                                        WHERE  action = 'cancel_order')) as canceled_orders, date_part('hour', creation_time)::int as hour
               FROM   orders
               GROUP BY hour
               ORDER BY hour) as t2 using(hour)) as t12
```